<a href="https://colab.research.google.com/github/PwCMoonshot/HR-Attrition-ML/blob/master/HRaaS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U bigchaindb-driver

    100% |████████████████████████████████| 512kB 5.5MB/s 
    100% |████████████████████████████████| 61kB 8.9MB/s 
    100% |████████████████████████████████| 133kB 5.2MB/s 
    100% |████████████████████████████████| 1.2MB 8.5MB/s 
    100% |████████████████████████████████| 122kB 13.0MB/s 
    100% |████████████████████████████████| 542kB 10.2MB/s 
    100% |████████████████████████████████| 2.1MB 10.0MB/s 
    100% |████████████████████████████████| 102kB 28.9MB/s 
  Running setup.py bdist_wheel for bigchaindb-driver ... - \ done
  Stored in directory: /root/.cache/pip/wheels/c5/45/fa/a173be482e245c04478338ac16a2f18b47e8f8c3ec1d6d7b36
  Running setup.py bdist_wheel for cryptoconditions ... - \ done
  Stored in directory: /root/.cache/pip/wheels/f8/0f/10/1c8fe7b487711e97fecc1e89eacec05c7c8adb008764ebf0ab
  Running setup.py bdist_wheel for python-rapidjson-schema ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/d8/39/b0/9178b7edad4c99363255da494f2ace20618c

In [0]:
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair
from time import sleep
from sys import exit

In [0]:
finance, marketing = generate_keypair(), generate_keypair()

bdb_root_url = 'https://test.bigchaindb.com'

bdb = BigchainDB(bdb_root_url)

LoganMcNeil = {
    'data': {
        'logan': {
            'wid': '3aa5550b7fe348b98d7b5741afc65534',
            'date_of_birth': '1971-05-25'
        },
    },
}

LoganMcNeil_metadata = {
    'position': 'Vice President, Human Resources'
}


In [0]:
prepared_create_tx = bdb.transactions.prepare(
        operation='CREATE',
        signers=finance.public_key,
        asset=LoganMcNeil,
        metadata=LoganMcNeil_metadata
    )

In [0]:
fulfilled_creation_tx = bdb.transactions.fulfill(
        prepared_create_tx,
        private_keys=finance.private_key
    )

In [0]:
sent_creation_tx = bdb.transactions.send_commit(fulfilled_creation_tx)

In [0]:
sent_creation_tx == fulfilled_creation_tx

True

In [0]:
txid = fulfilled_creation_tx['id']

In [0]:
print(txid)

37f1511f4a5c8ba0c8b61d8a64c3d593a8cc8729a96587b29ab6b796642e855a


In [0]:
block_height = bdb.blocks.get(txid=fulfilled_creation_tx['id'])
block = bdb.blocks.retrieve(str(block_height))
print(block)

{'height': 287311, 'transactions': [{'inputs': [{'owners_before': ['GpbcuZXm4hgEgTAFGJzsDCQZx3gjLLbeUTLkeZgwuQAY'], 'fulfills': None, 'fulfillment': 'pGSAIOsQvu7cFZRStdIZ0FsTMimax7FdpUJHaJvt6cyq9-LFgUCbw3zdwJAsbWZ66jv1rv1e1YPt-feL4ANM-DWcg_PDX5IyuK5AZkjTJKWrhwpVNUiMsTwRa_ztI6RVDKanTTsE'}], 'outputs': [{'public_keys': ['GpbcuZXm4hgEgTAFGJzsDCQZx3gjLLbeUTLkeZgwuQAY'], 'condition': {'details': {'type': 'ed25519-sha-256', 'public_key': 'GpbcuZXm4hgEgTAFGJzsDCQZx3gjLLbeUTLkeZgwuQAY'}, 'uri': 'ni:///sha-256;eAUMSMDpfKuMV7-FunNvP3gw3Mdh-sWNMIOAM0KvIBQ?fpt=ed25519-sha-256&cost=131072'}, 'amount': '1'}], 'operation': 'CREATE', 'metadata': {'position': 'Vice President, Human Resources'}, 'asset': {'data': {'logan': {'wid': '3aa5550b7fe348b98d7b5741afc65534', 'date_of_birth': '1971-05-25'}}}, 'version': '2.0', 'id': '37f1511f4a5c8ba0c8b61d8a64c3d593a8cc8729a96587b29ab6b796642e855a'}]}


In [0]:
creation_tx = fulfilled_creation_tx

asset_id = creation_tx['id']

transfer_asset = {
     'id': asset_id,
}

In [0]:
output_index = 0

output = creation_tx['outputs'][output_index]

transfer_input = {
     'fulfillment': output['condition']['details'],
     'fulfills': {
          'output_index': output_index,
          'transaction_id': creation_tx['id'],
      },
      'owners_before': output['public_keys'],
 }

prepared_transfer_tx = bdb.transactions.prepare(
     operation='TRANSFER',
     asset=transfer_asset,
     inputs=transfer_input,
     recipients=marketing.public_key,
 )

In [0]:
fulfilled_transfer_tx = bdb.transactions.fulfill(
     prepared_transfer_tx,
     private_keys=finance.private_key,
 )

fulfilled_transfer_tx['outputs'][0]['public_keys'][0] == marketing.public_key

True

In [0]:
sent_transfer_tx = bdb.transactions.send_commit(fulfilled_transfer_tx)

In [0]:
print("Is Jim in Marketing?",
    sent_transfer_tx['outputs'][0]['public_keys'][0] == marketing.public_key)

Is Jim in Marketing? True


In [0]:
print("Was Jim in Finance?",
    fulfilled_transfer_tx['inputs'][0]['owners_before'][0] == finance.public_key)

Was Jim in Finance? True


In [0]:
bdb.metadata.get(search='Administration Assistant')

[{'id': 'd117d761923d0cb4ad2a55418f22afea77f4e945771e04bb2045637b8e621137',
  'metadata': {'position': 'Administration Assistant'}}]

In [0]:
bdb.assets.get(search='Jim')

[]

In [0]:
print(fulfilled_transfer_tx)

{'inputs': [{'owners_before': ['GpbcuZXm4hgEgTAFGJzsDCQZx3gjLLbeUTLkeZgwuQAY'], 'fulfills': {'transaction_id': '37f1511f4a5c8ba0c8b61d8a64c3d593a8cc8729a96587b29ab6b796642e855a', 'output_index': 0}, 'fulfillment': 'pGSAIOsQvu7cFZRStdIZ0FsTMimax7FdpUJHaJvt6cyq9-LFgUA2D3XwVRoSrfk8XfLeVIRFwcgCVK8oDi5-ifvhhDcisdqW-gL9rT6L6-g7pY7DhuQAEE0JgrCXgrPCAxVSKO0P'}], 'outputs': [{'public_keys': ['BZSaawkk3En3Kpnqz4tU4btKHZrtJpwzjTjPGroidY5B'], 'condition': {'details': {'type': 'ed25519-sha-256', 'public_key': 'BZSaawkk3En3Kpnqz4tU4btKHZrtJpwzjTjPGroidY5B'}, 'uri': 'ni:///sha-256;zVwQ8LGOLU8aVB7miqq_Va-1fU6ccSK9qXq2abYLdvg?fpt=ed25519-sha-256&cost=131072'}, 'amount': '1'}], 'operation': 'TRANSFER', 'metadata': None, 'asset': {'id': '37f1511f4a5c8ba0c8b61d8a64c3d593a8cc8729a96587b29ab6b796642e855a'}, 'version': '2.0', 'id': '45d0b4aea9b5d62795fe19e40a432d928bef639d2b5fa5b7c54aa67031f3decd'}
